In [1]:
import math
import numpy
import pyaudio
import pylab as pl
import time
import wave
CHUNK = 1024

%pylab inline
SINE=True
SINE=False 
ABSINE=True
ABSINE=False 
TRIANGLE=True
TRIANGLE=False 
SQUARE= True 
SQUARE=False
SAWTOOTH=True
SAWTOOTH=False
BEATS=False
BEATS=True

Populating the interactive namespace from numpy and matplotlib


In [2]:
def sine(frequency, length, rate):
    length = int(length * rate)
    factor = float(frequency) * (math.pi * 2) / rate
    pl.figure(figsize=(15,5))
    wave=numpy.sin(numpy.arange(length) * factor)
    pl.plot(wave[:1000])
    pl.ylim(-1.05,1.05)
    return wave

def square(frequency, length, rate):
    length = int(length * rate)
    factor = float(frequency) * (math.pi * 2) / rate
    pl.figure(figsize=(15,5))
    wave=np.zeros(length)
    wave[(numpy.sin(numpy.arange(length) * factor)>0)]=1
    wave[(numpy.sin(numpy.arange(length) * factor)<0)]=-1

    pl.plot(wave[:1000])
    pl.ylim(-1.05,1.05)
    return wave

def absine(frequency, length, rate):
    length = int(length * rate)
    factor = float(frequency) * (math.pi * 2) / rate
    pl.figure(figsize=(15,5))
    wave=np.abs(numpy.sin(numpy.arange(length) * factor))
    pl.plot(wave[:1000])
    pl.ylim(-1.05,1.05)
    return wave


def triangle(frequency, length, rate):    
    length = int(length * rate)
    factor = (float(frequency) / rate)*(math.pi*2)
    indx1=np.array((numpy.cos(numpy.arange(length) * factor)[:-1]>0)*(numpy.cos(numpy.arange(length) * factor)[1:]<0))
    indx2=np.array((numpy.cos(numpy.arange(length) * factor)[:-1]<0)*(numpy.cos(numpy.arange(length) * factor)[1:]>0))
    pl.figure(figsize=(15,5))
    wave=numpy.zeros(length) 
    wave[indx1]=1.0
    wave[indx2]=-1.0
    x=np.arange(length)
    line=[]
    for i,w in enumerate(wave[indx1+indx2][:-1]):
        
        line.append( np.poly1d(np.polyfit([x[indx1+indx2][i],x[indx1+indx2][i+1]],\
                                           [wave[indx1+indx2][i],wave[indx1+indx2][i+1]],1))\
            (x[(x>=x[indx1+indx2][i])*(x<x[indx1+indx2][i+1])]))
        
    line=np.concatenate(line)
    pl.plot(line[:1000])
    pl.ylim(-1.05,1.05)
    return line

def sawtooth(frequency, length, rate):    
    length = int(length * rate)
    factor = (float(frequency) / rate)*(math.pi*2)
    indx1=np.array((numpy.cos(numpy.arange(length) * factor)[:-1]>0)*(numpy.cos(numpy.arange(length) * factor)[1:]<0))
    indx2=np.concatenate([indx1[1:],[False]])
    pl.figure(figsize=(15,5))
    wave=numpy.zeros(length) 
    wave[indx1]=1.0
    wave[indx1[1:]]=-1.0    
    x=np.arange(length)
    line=[]
    for i,w in enumerate(wave[indx1+indx2][:-1]):
        line.append( np.poly1d(np.polyfit([x[indx1+indx2][i],x[indx1+indx2][i+1]],\
                                           [wave[indx1+indx2][i],wave[indx1+indx2][i+1]],1))\
            (x[(x>=x[indx1+indx2][i])*(x<x[indx1+indx2][i+1])]))
        
    line=np.concatenate(line)
    pl.plot(line[:1000])
    pl.ylim(-1.05,1.05)
    return line

def observed():
    import pickle as pkl
    from scipy.interpolate import interp1d

    obs=pkl.load(open("test2_photometry.pkl","r"))
    line=interp1d(obs[0],obs[1],bounds_error=False,fill_value=0.0)(np.linspace(min(obs[0]),max(obs[0]),10000))
    pl.plot(line)
    pl.ylim(-1.05,1.05)
    return np.concatenate([[line]*2])
    
    
def beats(f1, f2, length, rate):
    frequency1=f1
    frequency2=f2
    length = int(length * rate)
    factor1 = float(frequency1) * (math.pi * 2) / rate 
    factor2 = float(frequency2) * (math.pi * 2) / rate
    pl.figure(figsize=(15,5))
    wave1=numpy.sin(numpy.arange(length) * factor1)
    wave2=numpy.sin(numpy.arange(length) * factor2)

    wave = (wave1+wave2)/2
    pl.plot(np.arange(48000)/48000.0,wave[:48000])
    pl.ylim(-1.05,1.05)
    pl.xlabel("seconds")
    
    print ("frequencies:", frequency1, frequency2)
    return wave

In [3]:
def play_tone(stream, frequency=340, length=10, rate=44100):
    if 0:
        chunks = []
        chunks.append(observed())
        chunks = numpy.concatenate(chunks) * 0.25

        stream.write(chunk.astype(numpy.float32).tostring())
        time.sleep(1.0)
    
    if SINE:
        chunks = []
        chunks.append(sine(frequency, length, rate))

        chunk = numpy.concatenate(chunks) * 0.25

        stream.write(chunk.astype(numpy.float32).tostring())
        time.sleep(1.0)
    if SQUARE:
        chunks = []
        chunks.append(square(frequency, length, rate))

        chunk = numpy.concatenate(chunks) * 0.25

        stream.write(chunk.astype(numpy.float32).tostring())
        time.sleep(1.0)
    if ABSINE:
        chunks = []
        chunks.append(absine(frequency, length, rate))

        chunk = numpy.concatenate(chunks) * 0.25

        stream.write(chunk.astype(numpy.float32).tostring())        
        time.sleep(1.0)
    if TRIANGLE:
        chunks = []
        chunks.append(triangle(frequency, length, rate))

        chunk = numpy.concatenate(chunks) * 0.25

        stream.write(chunk.astype(numpy.float32).tostring()) 
        time.sleep(1.0)
    if SAWTOOTH:
        chunks = []
        chunks.append(sawtooth(frequency, length, rate))

        chunk = numpy.concatenate(chunks) * 0.25

        stream.write(chunk.astype(numpy.float32).tostring())   
        time.sleep(1.0)
        
        
    if BEATS:
        chunks = []
        chunks.append(beats(frequency, 440, length, rate))

        chunk = numpy.concatenate(chunks) * 0.25

        stream.write(chunk.astype(numpy.float32).tostring())
        time.sleep(1.0)


In [34]:
import wave
RATE = 48000.
RECORD_SECONDS = 5.
CHUNK = 1024.
RECORD = False
RECORD = True
if __name__ == '__main__':
    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paFloat32,
                    channels=1, rate=int(RATE), 
                    input=True,output=2,
                    frames_per_buffer=int(CHUNK))
    print stream
    if RECORD:
        print "recording..."
        frames = []
 
        for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
            data = stream.read(int(CHUNK))
            frames.append(data)

        print "finished recording"
        stream.stop_stream()

 
# stop Recording
    else:
        print (stream)
        play_tone(stream, frequency = 440)#439.75)

    stream.close()
    p.terminate()
    #stream.read(256)

    
    waveFile = wave.open("test.wav", 'wb')
    waveFile.setnchannels(2)

    waveFile.setsampwidth(p.get_sample_size(pyaudio.paFloat32))
    waveFile.setframerate(int(RATE))        
    waveFile.writeframes(b''.join(frames))
    waveFile.close()

<pyaudio.Stream instance at 0x10ddabea8>
recording...
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished recording
finished record

In [30]:
print ""